In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mitochondrial_Disorders"
cohort = "GSE30933"

# Input paths
in_trait_dir = "../../input/GEO/Mitochondrial_Disorders"
in_cohort_dir = "../../input/GEO/Mitochondrial_Disorders/GSE30933"

# Output paths
out_data_file = "../../output/preprocess/Mitochondrial_Disorders/GSE30933.csv"
out_gene_data_file = "../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE30933.csv"
out_clinical_data_file = "../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE30933.csv"
json_path = "../../output/preprocess/Mitochondrial_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene expression data availability assessment
# Based on the background information, this study focuses on gene expression profiling in PBMCs from FRDA patients.
# The series title mentions "A Gene Expression Phenotype In Lymphocytes From Friedreich's Ataxia Patients"
is_gene_available = True

# 2.1 Data Availability
# Reviewing sample characteristics dictionary:
# Row 0: 'disease status' contains information about FRDA (a type of mitochondrial disorder), Carrier, or Normal status
trait_row = 0  # This contains the disease status information which matches our trait (Mitochondrial_Disorders)

# Age and gender information are not provided in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert disease status to binary format (1 for disease, 0 for non-disease)."""
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: FRDA (disease) = 1, Normal/Carrier = 0
    if value.lower() == "frda":
        return 1
    elif value.lower() in ["normal", "carrier"]:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to numerical value."""
    # Function defined but not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    # Function defined but not used since gender data is not available
    return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the matrix file
    matrix_file = os.path.join(in_cohort_dir, "GSE30933_series_matrix.txt.gz")
    
    # Parse the clinical data from the matrix file
    # Since the tools module is already imported, we can directly access its functions
    clinical_data = get_geo_clinical_data(matrix_file)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 4: Gene Identifier Review

In [ ]:
# Based on the identifiers, these are Illumina probes (ILMN_*) and not human gene symbols
# These probe IDs need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns for mapping
# Based on the preview, we can see:
# - Gene identifiers in expression data are in the 'ID' column (ILMN_* format)
# - Gene symbols are in the 'SYMBOL' column

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='SYMBOL')
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First few genes:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Re-load the gene expression data that was saved in a previous step
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Successfully loaded gene data from {out_gene_data_file}")
except Exception as e:
    print(f"Error loading gene data: {e}")
    raise

# 2. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")

# 3. Extract clinical data from the matrix file again to be sure we have the correct data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Based on the sample characteristics dictionary, the trait information is in row 0 (disease status)
# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert FRDA disease status to binary (1 = FRDA, 0 = Normal or Carrier)"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "frda":
        return 1
    elif value.lower() in ["normal", "carrier"]:
        return 0
    else:
        return None

# Create the clinical dataframe using the correct trait row
trait_row = 0  # Row for disease status (FRDA)
is_trait_available = True

try:
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,  # Using the predefined trait variable
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,    # Age information not available
        convert_age=None,
        gender_row=None,  # Gender information not available
        convert_gender=None
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait is biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from human samples with Friedreich's Ataxia (FRDA). The trait variable indicates FRDA status (1=FRDA, 0=Normal/Carrier)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=is_trait_available,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # Make sure to properly handle validation in the exception case
    df_empty = pd.DataFrame()
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=df_empty,  # Empty DataFrame
        note="Failed to extract or process clinical data, but gene expression data is available."
    )
    print("Dataset validation failed due to clinical data processing errors. Gene data was saved.")